In [1]:
import sys
import os
import pandas as pd
import numpy as np

from bs4 import BeautifulSoup
from selenium import webdriver
import time
import tqdm
from tqdm.notebook import tqdm

In [2]:
# "url_list.csv" 불러오기
url_load = pd.read_excel("blog crawl.xlsx")        # 기본 모델
num_list = len(url_load)  

print(num_list)
url_load

100


,Unnamed: 0,url,title
0,0,https://blog.naver.com/shfovkeh?Redirect=Log&l...,"[EPL 프리뷰] 연승 끊긴 맨유, 동기부여 잃은 크리스탈팰리스를 만나다! (크팰 ..."
1,1,http://blog.dhlkorea.com/222031760679,2020/21 시즌 맨유의 홈 유니폼 받아 가세요!
2,2,https://blog.naver.com/wjssha?Redirect=Log&log...,[2019-20 36R: 맨유 - 크리스탈 팰리스] 크리스탈 팰리스 경기 스타일 분...
3,3,https://blog.naver.com/boohe97?Redirect=Log&lo...,[19-20 FA컵 4강 리뷰] 맨유 vs 첼시
4,4,https://blog.naver.com/king978?Redirect=Log&lo...,"[vs 사우스햄튼] 기회를 놓친 맨유, 언성 히어로 집단 사우스햄튼"
...,...,...,...
95,95,https://blog.naver.com/rlaqnska7?Redirect=Log&...,케인 맨유 원해
96,96,https://blog.naver.com/totti0502?Redirect=Log&...,"'포스트 퍼거슨' 맨유 솔샤르, 장기집권 선결 과제는?"
97,97,https://blog.naver.com/totti0502?Redirect=Log&...,"'결정적 실수·뒷심 부족' 맨유, 4위권 진입 놓친 통한의 무승부"
98,98,https://blog.naver.com/lth2209?Redirect=Log&lo...,맨유의 챔피언스리그 진출 확률에 대해서


In [3]:
dict = {}  # 전체 크롤링 데이터를 담을 그릇

number = 100 # 수집할 글 갯수 (엑셀 파일에서 글 갯수와 비교해서 자신이 수집할 갯수를 선정)
for i in tqdm(range(0, number)): 
    # 글 띄우기
    url = url_load['url'][i]   # url은 엑셀에서 url 이라는 제목을 기준으로 사이트를 크롤링했고 이를 다시 띄우기 위해 사용 
    driver = webdriver.Chrome("C:/Users/USER/Desktop/python/chromedriver")
    driver.get(url)   # 글 띄우기
    
    # 사이트에서 글 크롤링 try, except는 100개라는 크롤링을 하던 중 일부 진행하다가 error가 날 경우를 대비하기 위해서 사용 
    try : 
        # iframe 접근
        driver.switch_to_frame('mainFrame')
        target_info = {}

        # 제목 크롤링 시작
        overlays = ".se-fs-.se-ff-" # 제목이 가지고 있는 사이트 html의 element                                 
        tit = driver.find_element_by_css_selector(overlays)  # 제목이 가지고 있는 사이트 element   
        title = tit.text

        # 글쓴이 크롤링 시작
        overlays = ".nick"         # 글쓴이가 가지고 있는 사이트 html의 element
        nick = driver.find_element_by_css_selector(overlays)         
        nickname = nick.text

        # 날짜 크롤링
        overlays = ".se_publishDate.pcol2"      # 날짜가 가지고 있는 사이트 html의 element                           
        date = driver.find_element_by_css_selector(overlays)        
        datetime = date.text

        # 내용 크롤링
        overlays = ".se-component.se-text.se-l-default"    # 내용이 가지고 있는 사이트 html의 element                                
        contents = driver.find_elements_by_css_selector(overlays)        

        content_list = []             # 내용의 경우 블로그 마다 장문으로 여러 줄로 이루어져있기에 리스트를 만들어서 하나로 합치는 작업을 해야함
        for content in contents:
            content_list.append(content.text)

        content_str = ' '.join(content_list)    # .join은 문장을 하나로 합치는 함수

        # 전체 100개의 게시글 중에 글 하나를 target_info라는 딕셔너리에 담고 이를 전체적으로 dict라는 딕셔너리에 담는 다
        # target_info 딕셔너리에 title ~ content라는 key 값으로 데이터들이 담김
        target_info['title'] = title
        target_info['nickname'] = nickname
        target_info['datetime'] = datetime
        target_info['content'] = content_str

        # 각각의 글을 dict라는 딕셔너리에 담기게 됩니다.
        dict[i] = target_info
        time.sleep(1)  # time.sleep을 해주면서 웹 브라우저를 켜고 닫는 순간을 나누기 위해서 사용
        
        print(i, title)

        # 글 하나 크롤링 후 크롬 창 닫기 (for문을 반복하여 크롤링시 새로운창이 계속 뜨게 되기에 닫아준다)
        driver.close()       
    
    # 에러나면 현재 크롬창 닫고 다음 글(i+1)로 이동
    except:
        driver.close()
        time.sleep(1)
        continue
    
    # 글 30, 50, 80 번째 마다 엑셀로 저장 (데이터 중간, 중간에 파일로 저장하기 위해서 함)
    if i == 30 or 50 or 80:
        result_df = pd.DataFrame.from_dict(dict, 'index')

        # 저장하기
        result_df.to_excel("blog_내용 크롤링.xlsx")
        time.sleep(3)

print('수집한 글 갯수: ', len(dict))
print(dict)# 수집한 글 개수 확인

result_df = pd.DataFrame.from_dict(dict, 'index')

# 저장하기
result_df.to_excel("blog_내용 크롤링.xlsx")

<ipython-input-3-b5603e9331d5>:13: DeprecationWarning: use driver.switch_to.frame instead
  driver.switch_to_frame('mainFrame')


0 [EPL 프리뷰] 연승 끊긴 맨유, 동기부여 잃은 크리스탈팰리스를 만나다! (크팰 대 맨유_SPOTV중계)
1 2020/2021 시즌 맨체스터 유나이티드 홈 유니폼 이벤트
2 2019-20 프리미어리그 36R
3 2차 창작 서보원
4 [vs 사우스햄튼] 기회를 놓친 맨유, 언성 히어로 집단 사우스햄튼
5 마이크 딘(희대의 🐕)
6 맨시티 징계 취소, 챔스 유로파 리그 진출 경쟁 점임가경 맨유 첼시 토트넘 남은 경기 일정은?
7 2005년의 박지성이 그립다.(feat.맨유동료들)
8 [Memory]98-99 맨유 VS 바이에른 뮌헨(UCL 결승)
9 대전시티 수원FC 최종검토
10 최근 5연승 순항 중이던 맨유를 막아냈던 사우샘프턴의 '전방 압박'
11 맨유와 첼시의 FA컵 준결승 프리뷰 (맨체스터 유나이티드)
12 UEFA 챔피언스리그 진출 경우의 수!모든 경우를 분석해보자(Feat. 첼시, 레스터, 맨유)
14 EPL 유망주 맨유이적설 있었던 벨링엄 분데스리가 도르트문트로 이적 확정
16 나단 아케
17 
18 이글은 읽으시면 더이상은 혼자서 힘들어 하시지 않아도 됩니다.
19 [19-20 FA컵 4강전]
20 엄청난 퍼포먼스를 보이고 있는 포르노(?) 조합
22 JAMES DAVIS
23 슈퍼세이브는 많은데 평범한 슈팅에서 실수가 너무 많다
24 크리스탈팰리스와 맨유의 프리미어리그 36라운드 프리뷰 (크리스털 팰리스 맨체스터 유나이티드)
25 [FA컵 맨유-첼시] 최고의 라이벌 매치
26 FA컵 4강 첼시 vs 맨유 / 올리비에 지루가 곧 전술이다
28 2020/07/20 맨유 vs 첼시 : 맨체스터'유리'나이티드
30 영국뉴스번역- 첼시 맨유를 제압하고 FA 컵 결승
31 [PL] 크리스탈팰리스 맨유 : 15년 째 맨유 팬인 사람의 이번 경기 예상
32 1960년대 초부터 가장 화려한 버즈비와 아이들의 시대가 시작됐다. 보비 찰턴 · 데니스 로 · 조지 베스트는 엄청난 위력을 선보이며 잉글랜드와 유럽 무대를 제패하기 시작했고, 여기에 버즈비 감독의 지

In [4]:
result_df.head(10)

,title,nickname,datetime,content
0,"[EPL 프리뷰] 연승 끊긴 맨유, 동기부여 잃은 크리스탈팰리스를 만나다! (크팰 ...",훈\n(shfovkeh),2020. 7. 15. 19:35,챔피언스리그 진출을 노리고 있는 맨유가\n이미 잔류가 확정되어 동기부여를 잃은\n크...
1,2020/2021 시즌 맨체스터 유나이티드 홈 유니폼 이벤트,DHL,2020. 7. 20. 10:19,2011년부터 DHL은 맨체스터 유나이티드의 공식 물류 파트너로 활동하고 있습니다....
2,2019-20 프리미어리그 36R,TSS\n(wjssha),2020. 7. 16. 12:05,지난 사우스햄튼 경기를 2-2로 비기면서 맨유는 챔피언스리그 진출권 도약에 실패했다...
3,2차 창작 서보원,마타롱,2020. 7. 20. 5:37,1. 경기 정보\n2020.07.20. 월 02:00 (한국 시간)\n웸블리 스타디...
4,"[vs 사우스햄튼] 기회를 놓친 맨유, 언성 히어로 집단 사우스햄튼",Hakang,2020. 7. 17. 8:48,"# 하센휘틀, 언성히어로 어벤져스를 만들었나\n참 유감스럽고 아쉬운 장면이 많았던 ..."
5,마이크 딘(희대의 🐕),우승길만걷자,2020. 7. 20. 5:41,"맨유와 첼시의 FA컵 준결승 경기가 마무리됐습니다. 첼시가 올리비에 지루, 메이슨 ..."
6,"맨시티 징계 취소, 챔스 유로파 리그 진출 경쟁 점임가경 맨유 첼시 토트넘 남은 경...",J Yellow,2020. 7. 16. 14:06,맨시티가 UEFA의 FFP 위반 징계를 피했습니다.\n이로서 2위를 확정지은 맨시티...
7,2005년의 박지성이 그립다.(feat.맨유동료들),기억\n(hememo),2020. 7. 20. 20:05,2005-06년 시즌은 박지성이 맨유에 입단한 첫 해다. 맨유에서 뛸때는 정말 지구...
8,[Memory]98-99 맨유 VS 바이에른 뮌헨(UCL 결승),조축,2020. 7. 19. 21:52,1999년 5월 26일 바르셀로나의 홈구장 캄프 누에서는 영화와 같은 일이 벌어졌다...
9,대전시티 수원FC 최종검토,은단이\n(br6c7c6vs),2020. 7. 19. 9:10,아무리 혼자서 애써봐도 제자리 걸음일뿐 입니다.\n지금이라도 늦지 않았습니다 저희와...
